<a href="https://colab.research.google.com/github/pacomm5/curso-series-temporales/blob/master/11.%20Auto%20Arima/Python/15_Auto_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [1]:
!pip install arch

     |████████████████████████████████| 902 kB 3.9 MB/s 
     |████████████████████████████████| 9.8 MB 39.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
!pip install pmdarima

     |████████████████████████████████| 1.4 MB 3.9 MB/s 


In [3]:
!pip install yfinance

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 9.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [5]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2020-03-20", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [6]:
df_comp = raw_data.copy()

In [7]:
df_comp.head()

^GDAXI               ...         ^N225       
                   Open         High  ...         Close Volume
Date                                  ...                     
1994-01-07  2218.959961  2227.639893  ...  18124.009766    0.0
1994-01-10  2231.840088  2238.010010  ...  18443.439453    0.0
1994-01-11  2225.429932  2235.610107  ...  18485.250000    0.0
1994-01-12  2227.120117  2227.790039  ...  18793.880859    0.0
1994-01-13  2171.500000  2183.709961  ...  18577.259766    0.0

[5 rows x 20 columns]

In [9]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]
# vamos a coger solo el precio de cierre

In [10]:
df_comp.head()

^GDAXI               ...         ftse        nikkei
                   Open         High  ...                           
Date                                  ...                           
1994-01-07  2218.959961  2227.639893  ...  3446.000000  18124.009766
1994-01-10  2231.840088  2238.010010  ...  3440.600098  18443.439453
1994-01-11  2225.429932  2235.610107  ...  3413.800049  18485.250000
1994-01-12  2227.120117  2227.790039  ...  3372.000000  18793.880859
1994-01-13  2171.500000  2183.709961  ...  3360.000000  18577.259766

[5 rows x 24 columns]

In [11]:
df_comp.tail()

^GDAXI               ...         ftse        nikkei
                   Open         High  ...                           
Date                                  ...                           
2020-03-13  9480.780273  9985.740234  ...  5366.100098  17431.050781
2020-03-16  8728.480469  8967.110352  ...  5151.100098  17002.039062
2020-03-17  9141.169922  9145.929688  ...  5294.899902  17011.529297
2020-03-18  8613.349609  8670.410156  ...  5080.600098  16726.550781
2020-03-19  8495.940430  8668.480469  ...  5151.600098  16552.830078

[5 rows x 24 columns]

In [12]:
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [13]:
df_comp.head()

,spx,dax,ftse,nikkei
,,,,
Date,,,,
1994-01-07,469.899994,2224.949951,3446.000000,18124.009766
1994-01-10,475.269989,2225.000000,3440.600098,18443.439453
1994-01-11,474.130005,2228.100098,3413.800049,18485.250000
1994-01-12,474.170013,2182.060059,3372.000000,18793.880859
1994-01-13,472.470001,2142.370117,3360.000000,18577.259766


### Creating Returns

In [15]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100
# aquí calculamos los retornos.

### Splitting the Data

In [17]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [18]:
from pmdarima.arima import auto_arima

In [19]:
model_auto = auto_arima(df.ret_ftse[1:])

In [20]:
model_auto # nos dice que coge ARIMA de orden 4,0,5 

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True)

In [21]:
model_auto.summary()
# modelo sarimax 4,0,5 pero no tiene la parte exogena. tampoco tiene la parte s estacional. realmente es un arma 4,5
# valora en función de AIC y no la significación de p valor.

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5467
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -8450.890
Date:                Sat, 19 Feb 2022   AIC                          16923.780
Time:                        23:21:29   BIC                          16996.452
Sample:                             0   HQIC                         16949.136
                               - 5467                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0291      0.023      1.269      0.204      -0.016       0.074
ar.L1          0.0164      0.080      0.204      0.839      -0.141       0.174
ar.L2         -0.6501      0.077     -8.419      0.000      -0.801      -0.499
ar.L3         -0.1682      0.071     -2.363      0.018      -0.308      -0.029
ar.L4          0.2130      0.075      2.843      0.004       0.066       0.360
ma.L1         -0.0392      0.080     -0.489      0.625      -0.196       0.118
ma.L2          0.6047      0.078      7.754      0.000       0.452       0.758
ma.L3          0.0748      0.069      1.083      0.279      -0.061       0.210
ma.L4         -0.2082      0.074     -2.820      0.005      -0.353      -0.063
ma.L5         -0.0993      0.009    -11.109      0.000      -0.117      -0.082
sigma2         1.2889      0.013     96.328      0.000       1.263       1.315
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              7482.28
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               1.81   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [ ]:
# aquí le mete parametros. 
model_auto2 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample_size = int(len(df_comp)*0.2))


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [ ]:
model_auto2.summary()
# aquí ya hay un orden estacional. 
# los tres coeficienetes de las partes exojenas. 
# la verosimilitud es mejor que le anterior. 
# el BIC ha ha bajado bastante. 

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                
=========================================================================================
Dep. Variable:                                 y   No. Observations:                 6834
Model:             SARIMAX(0, 0, 1)x(4, 0, 0, 5)   Log Likelihood               -6809.273
Date:                           Fri, 24 Apr 2020   AIC                          13640.547
Time:                                   10:53:15   BIC                          13715.673
Sample:                                        0   HQIC                         13666.465
                                          - 6834                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0008      0.015     -0.052      0.959      -0.031       0.029
drift      -2.401e-06   4.33e-06     -0.555      0.579   -1.09e-05    6.08e-06
x1             0.0872      0.006     15.219      0.000       0.076       0.098
x2             0.5624      0.005    110.721      0.000       0.552       0.572
x3             0.0720      0.004     16.452      0.000       0.063       0.081
ma.L1         -0.1165      0.008    -15.498      0.000      -0.131      -0.102
ar.S.L5       -0.0332      0.009     -3.776      0.000      -0.050      -0.016
ar.S.L10      -0.0503      0.010     -5.225      0.000      -0.069      -0.031
ar.S.L15      -0.0349      0.009     -3.844      0.000      -0.053      -0.017
ar.S.L20      -0.0233      0.009     -2.521      0.012      -0.041      -0.005
sigma2         0.4641      0.005     99.300      0.000       0.455       0.473
===================================================================================
Ljung-Box (Q):                       76.91   Jarque-Bera (JB):             15206.90
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.48   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.29
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""